# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

  - 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies

We'll first install all our required libraries.

In [1]:
!pip install -qU langchain langchain_openai langchain-community langgraph arxiv duckduckgo_search==5.3.1b1

## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE4 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Duck Duck Go Web Search](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/ddg_search)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

####🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [4]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_belt = [
    ### YOUR CODE HERE☺
    DuckDuckGoSearchRun(),
    ArxivQueryRun()
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [6]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?

--> The model determines which tool to use based on relevance assessment, parameter matching, and efficiency.

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [7]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [8]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [9]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [10]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [11]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [12]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [13]:
compiled_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

--> We could end up cycling indefinitely, which is why the graph needs to have an end or exit strategy. One way to do this is to provide clear instructions with a conditional edge with a path to either continue or end the cycle. Once the condition to end has been met, the cycle ends. 

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [14]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_toTgvVOpG9fO1k3hj5UmqTVB', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets 2023"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 156, 'total_tokens': 181}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c9aa9c0491', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ff1d1f00-86b6-482b-8857-eb27b30ab9df-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current captain of the Winnipeg Jets 2023'}, 'id': 'call_toTgvVOpG9fO1k3hj5UmqTVB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 156, 'output_tokens': 25, 'total_tokens': 181})]



Receiving update from node: 'action'
[ToolMessage(content='Posted September 12, 2023 9:29 am. Centre Adam Lowry was named the Winnipeg Jets new captain on Tuesday. Lowry is the third Je

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [15]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using DuckDuckGo.")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hhwbA7cV7OzK5q7CheUi0usp', 'function': {'arguments': '{"query": "QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_QJHUr6wPavclHwBFWrtt8WN9', 'function': {'arguments': '{"query": "latest Tweet Tim Dettmers"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_OyScPdbU6I1impKZa45Zvvtb', 'function': {'arguments': '{"query": "latest Tweet Artidoro Pagnoni"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_gQZ44zyY4hGPQAZ1h085962m', 'function': {'arguments': '{"query": "latest Tweet Teven Le Scao"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_0jRZn6F1GvjXOwLy7ovvMosw', 'function': {'arguments': '{"query": "latest Tweet Younes Belkada"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_9bNvAstSgjr6uanRuAONQUqb', 'function': {'arguments': '{"query": "latest Tweet Alexander M. Rush"}', 'name': 'duckduckgo_sear

####🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

1. State object received the request
2. State passed to agent where the AIMessage was added 
3. Agent passed message to the conditional edge and proceeded to search archix for the authors
4. Agent added the response to the object and continued to the conditional edge
5. Agent perfomed a duckduckgo search term of "latest tweet of [author]" 
6. Agent added the response to the object and continued to the conditional edge
7. Conditional edge did not find any more tools to call, so the condition of "end" was met


## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [16]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | compiled_graph | parse_output

In [17]:
agent_chain.invoke({"question" : "What is RAG?"})

"RAG stands for Retrieval-Augmented Generation. It is a technique used in natural language processing (NLP) and machine learning to improve the performance of language models by combining retrieval-based methods with generative models. Here's a brief overview of how it works:\n\n1. **Retrieval**: In the first step, the system retrieves relevant documents or pieces of information from a large corpus based on the input query. This is typically done using a retrieval model, such as BM25 or a dense retrieval model like DPR (Dense Passage Retrieval).\n\n2. **Augmentation**: The retrieved documents are then used to augment the input query. This can involve concatenating the retrieved documents with the original query or using them to provide additional context.\n\n3. **Generation**: Finally, a generative model, such as a transformer-based model like GPT-3, is used to generate a response based on the augmented input. The generative model can produce more accurate and contextually relevant res

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

####🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [36]:
questions = [
    "How does QLoRA backpropagates gradients?",
    "What does QLoRA stand for?",
    "What are the key components of the QLoRA method?",
    "What are the performance implications of using QLoRA compared to traditional fine-tuning?",
    "Why was QLoRA developed?"
]

answers = [
    {"must_mention" : ["frozen", "4-bit quantized"]},
    {"must_mention" : ["Quantized", "Adaptation"]},
    {"must_mention" : ["Quantization", "LoRA"]},
    {"must_mention" : ["efficient", "fine-tuning"]},
    {"must_mention" : ["computational", "costs"]}
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [39]:
from langsmith import Client

client = Client()
dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

--> The correct answers are associated based on their position in the list. The create_examples method also lists outputs as optional. Both of these could be problematic if the answers are out of order or if there aren't enough/any answers given.

### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [40]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

--> Gaps exist in this method with the text matching. This is problematic if the answers don't have the correct capitalization, spelling, or punctuation.

Now that we have created our custom evaluator - let's initialize our `RunEvalConfig` with it!

In [41]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[must_mention],
)

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [42]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=agent_chain,
    evaluation=eval_config,
    verbose=True,
    project_name=f"RAG Pipeline - Evaluation - {uuid4().hex[0:8]}",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'RAG Pipeline - Evaluation - 1e16ad1c' at:
https://smith.langchain.com/o/dc2f5571-a21b-5103-9ccb-c42bfdb936b0/datasets/cbee13a3-d7aa-4980-9c0d-f47bf9a7c4d4/compare?selectedSessions=f8def839-e339-40b5-846b-678bd5017c0a

View all tests for Dataset Retrieval Augmented Generation - Evaluation Dataset - a72bd0c9 at:
https://smith.langchain.com/o/dc2f5571-a21b-5103-9ccb-c42bfdb936b0/datasets/cbee13a3-d7aa-4980-9c0d-f47bf9a7c4d4
[------------------------------------------------->] 5/5

feedback.must_mention error  execution_time  \
count                      5     0        5.000000   
unique                     2     0             NaN   
top                     True   NaN             NaN   
freq                       4   NaN             NaN   
mean                     NaN   NaN        6.226795   
std                      NaN   NaN        2.624612   
min                      NaN   NaN        3.286224   
25%                      NaN   NaN        4.127506   
50%                      NaN   NaN        6.766388   
75%                      NaN   NaN        7.048113   
max                      NaN   NaN        9.905743   

                                      run_id  
count                                      5  
unique                                     5  
top     8899a1d6-36ad-4209-a74a-e1e9fc642ee6  
freq                                       1  
mean                                     NaN  
std                                      NaN  
min                                      NaN  
25%                                      NaN  
50%                                      NaN  
75%                                      NaN  
max                                      NaN

{'project_name': 'RAG Pipeline - Evaluation - 1e16ad1c',
 'results': {'f122ca72-2f9f-441b-b1cb-17ef24663545': {'input': {'question': 'How does QLoRA backpropagates gradients?'},
   'feedback': [EvaluationResult(key='must_mention', score=False, value=None, comment=None, correction=None, evaluator_info={}, feedback_config=None, source_run_id=UUID('436e98a7-56c9-4083-a137-172e223021e4'), target_run_id=None)],
   'execution_time': 6.766388,
   'run_id': '8899a1d6-36ad-4209-a74a-e1e9fc642ee6',
   'output': "QLoRA (Quantized Low-Rank Adaptation) is a technique used to fine-tune large language models efficiently by reducing the computational and memory requirements. The key idea behind QLoRA is to use quantization and low-rank adaptation to make the fine-tuning process more efficient.\n\nHere's a high-level overview of how QLoRA backpropagates gradients:\n\n1. **Quantization**: The model weights are quantized to reduce the memory footprint. This involves converting the weights from floating-p

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [43]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

####🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.

##### YOUR MARKDOWN HERE

--> The code sets up a state graph with two nodes: one for the agent's model calling behavior ("agent") and another for the agent's actions ("action"). The graph structure will allow transitions between these states, managing how the agent operates based on the defined nodes and their associated functions. This setup could be part of a more extensive system where agents perform tasks based on state transitions in a defined workflow or decision-making process.

In [44]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

##### YOUR MARKDOWN HERE
--> By setting the "agent" node as the entry point, the state graph will begin its operations from this node. This is crucial in workflows or state machines where you need to define a clear starting point for the sequence of actions or state transitions. In practical terms, when the agent starts operating within this graph, it will first invoke the behavior or function associated with the "agent" node (call_model in this case) before potentially moving on to other nodes like "action".

In [45]:
graph_with_helpfulness_check.set_entry_point("agent")

##### YOUR MARKDOWN HERE

--> This function tool_call_or_helpful is designed to evaluate the flow of a conversation or interaction and decide what the next step should be. It does this by:

1. Checking if a tool was used.
2. Assessing the length of the conversation.
3. Using a language model (GPT-4) to evaluate the helpfulness of the final response based on the initial query.

Depending on these checks, the function will decide whether to continue the interaction, take a specific action, or end the conversation.

In [46]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4")

  helpfulness_chain = prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

####🏗️ Activity #4:

Please write what is happening in our `tool_call_or_helpful` function!
--> The function evaluates the state of a conversation or interaction and determine the next step based on the messages exchanged. The function integrates with a language model (in this case, OpenAI's GPT-4) to assess whether a final response is helpful.

##### YOUR MARKDOWN HERE

--> This line of code is configuring the state graph to make decisions dynamically based on the outcomes of the tool_call_or_helpful function. Here's what happens:

The graph starts at the "agent" node.
It calls the tool_call_or_helpful function, which examines the current state (like the last message, tool calls, and helpfulness).
Based on the function's return value:
If "continue", the graph stays at the "agent" node.
If "action", the graph moves to the "action" node.
If "end", the graph transitions to the END node, concluding the process.
This setup allows the graph to handle complex interactions and determine the flow of actions based on the logic encapsulated in tool_call_or_helpful, making the system responsive and adaptive to the current state of the conversation or task.


In [47]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### YOUR MARKDOWN HERE
--> By adding this edge, you are defining that once the graph reaches the "action" node and completes whatever process or tool invocation is associated with that state, it will move back to the "agent" node. This setup suggests that the process or system represented by this graph is iterative or cyclical, potentially allowing for multiple actions or evaluations to be performed in sequence, with the graph looping between the "agent" and "action" nodes as needed.

This kind of loop is common in workflows or state machines where actions might be repeated or where the system needs to continuously check or process new inputs after performing an action.

In [48]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### YOUR MARKDOWN HERE
--> The code agent_with_helpfulness_check = graph_with_helpfulness_check.compile() takes the state graph you’ve defined with nodes, edges, and conditional logic, and compiles it into an optimized, executable form. The resulting agent_with_helpfulness_check object is now a fully operational state machine or workflow that can be used to guide an agent's behavior, making decisions based on the inputs and conditions you've set up in the graph.

This is a crucial step when you’re ready to move from defining how the agent should behave to actually deploying and using that behavior in practice.

In [32]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### YOUR MARKDOWN HERE
--> This code is designed to interact with the state graph (agent_with_helpfulness_check) in an asynchronous manner, processing the input messages in a streaming fashion. As the graph processes the input, updates from different nodes are streamed back and printed out. This setup allows for real-time tracking of how the state graph is handling the input, showing what each node contributes to the overall processing of the message.

The use of asynchronous processing and streaming is particularly useful when dealing with complex workflows or interactions that may take time to complete, as it allows you to see partial results as they become available, rather than waiting for the entire process to finish.

In [49]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wlkqQJSOMCqAwr2whdAK62Hb', 'function': {'arguments': '{"query": "LoRA machine learning"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_E64rUhQ1rKwiRVY4wHV0xJAw', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_4SREmXGsUFinmng802NlXZEF', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 171, 'total_tokens': 247}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_fde2829a40', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7a07e0a3-4b1f-43c0-bd29-a5127f51bf70-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'LoRA machine learning'}, 'id': 'call_wlkqQJSOMCqAwr2whdAK62Hb', 'type': 'tool_call'}, 

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [34]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [35]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Prompt engineering is a concept primarily associated with the field of artificial intelligence, particularly in the context of natural language processing (NLP) and large language models (LLMs) like GPT-3. It involves the design and crafting of prompts (input text) to elicit desired responses from AI models. The goal is to optimize the input to get the most accurate, relevant, or useful output from the model.

### Key Aspects of Prompt Engineering:
1. **Crafting Effective Prompts**: Designing prompts that guide the AI to produce the desired output.
2. **Understanding Model Behavior**: Knowing how different models respond to various types of input.
3. **Iterative Testing**: Continuously refining prompts based on the responses received.
4. **Contextual Awareness**: Ensuring that the prompt provides enough context for the model to understand the task.

### When Did It Break Onto the Scene?
Prompt engineering became particularly prominent with the advent of large-scale language models like